## This is the notebook where the final benchmark values used as comparisons with our solution were computed.

### Imports

In [ ]:
%pip install quantum-isl
%pip install qiskit==0.34.0

In [ ]:
backend_names = {"fake_backend" : "aer_simulator(fake_jakarta)",
            "jakarta" : "ibmq_jakarta"}

In [ ]:
# Importing standard Qiskit modules
from qiskit.result import Result

# Import python scripts
import mitigation_utils 
import analysis_utils 
import file_utils

import numpy as np

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

### Obtaining comparisons for final benchmark.

Let's also see how accurate the unrecompiled solution is with / without measurement calibration:

In [ ]:
trotter_steps = 35
target_time = np.pi
num_qubits = 7

In [ ]:
backend = backend_names["fake_backend"]
print('Running on: {}'.format(backend))
shots = 16192
reps = 8

f_unrecomp_no_meas_cal = file_utils.get_partial_trotter_stomo_circuits_results_circuits_filename('0','pi', trotter_steps, trotter_steps, backend, shots, reps) 
results_circuits = np.load(f_unrecomp_no_meas_cal + '.npy', allow_pickle=True)

results_for_each_rep = results_circuits[0]
st_qcs = results_circuits[1]
fidelities = [analysis_utils.state_tomo(result, st_qcs) for result in results_for_each_rep] 
    
print('Time {}, backend {}, Trotter steps {}, shots {}, reps {}'.format(target_time, backend, trotter_steps, shots, reps))
print('Unrecompiled solution: fidelity {} \u00B1 {:.4f}'.format(np.mean(fidelities), np.std(fidelities)))


In [ ]:
backend = backend_names["fake_backend"]
print('Running on: {}'.format(backend))
shots = 16192
reps = 8

f_unrecomp = file_utils.get_partial_trotter_stomo_circuits_results_circuits_meas_cal_filename('0','pi', trotter_steps, trotter_steps, backend, shots, reps) 
results_circuits = np.load(f_unrecomp + '.npy', allow_pickle=True)

num_meas_cal_circuits = 8
fidelities = []

results_for_each_rep = results_circuits[0]
st_qcs = results_circuits[1]

for rep in range(reps):
    results = results_for_each_rep[rep]
    corrected_results = []
        
    for i in range(0, len(results.results), 3 + num_meas_cal_circuits):
        stomo_results = results.results[i : i+3]
        meas_cal_results = results.results[i+3 : (i+3) + num_meas_cal_circuits]

        cmf = mitigation_utils.do_measurement_calibration(backend, num_qubits, [1,3,5], cal_results=Result(backend, 'backend version', 'QOBJ???', 'JOB_ID???', True, meas_cal_results))
        corrected_stomo_results = cmf.filter.apply(Result(backend, 'backend version', 'QOBJ???', 'JOB_ID???', True, stomo_results)) 
        corrected_results += corrected_stomo_results.results
    
    fidelities.append(analysis_utils.state_tomo(Result(backend, 'backend version', 'QOBJ???', 'JOB_ID???', True, corrected_results), st_qcs))

print('Time {}, backend {}, Trotter steps {}, shots {}, reps {}'.format(target_time, backend, trotter_steps, shots, reps))
print('Unrecompiled solution with measurement calibration: fidelity {} \u00B1 {:.4f}'.format(np.mean(fidelities), np.std(fidelities)))


Recompiled solution with no measurement calibration.

In [ ]:
backend = backend_names["jakarta"]
print('Running on: {}'.format(backend))
shots = 8192
reps = 8
sufficient_cost = 1e-3

f = file_utils.get_partial_trotter_stomo_circuits_results_circuits_isl_filename('0','pi', trotter_steps, trotter_steps, backend, shots, reps, sufficient_cost) 
results_circuits = np.load(f + '.npy', allow_pickle=True)

results = results_circuits[0]
st_qcs = results_circuits[1]
fidelities = [analysis_utils.state_tomo(result, st_qcs) for result in results] 

print('Time {}, backend {}, Trotter steps {}, shots {}, reps {}, ISL suff cost {}'.format(target_time, backend, trotter_steps, shots, reps, sufficient_cost))
print('Fidelity {} \u00B1 {:.4f}'.format(np.mean(fidelities), np.std(fidelities)))
